# Sales Data Warehouse Implementation in Microsoft Fabric

**Author:** [Your Name]  
**Date:** February 2025  
**Platform:** Microsoft Fabric  

---

## Table of Contents
1. [Project Overview](#1-project-overview)
2. [Architecture](#2-architecture)
3. [Implementation](#3-implementation)
   - 3.1 [Schema Creation](#31-schema-creation)
   - 3.2 [Table Creation](#32-table-creation)
   - 3.3 [Constraints & Relationships](#33-constraints--relationships)
   - 3.4 [Data Loading](#34-data-loading)
4. [Analytics & Insights](#4-analytics--insights)
   - 4.1 [Top Customers Analysis](#41-top-customers-analysis)
   - 4.2 [Top Products Analysis](#42-top-products-analysis)
   - 4.3 [Category Leaders Analysis](#43-category-leaders-analysis)
5. [Key Results](#5-key-results)
6. [Conclusion](#6-conclusion)

---

## 1. Project Overview

### Business Problem
As our business grows, we need a structured way to store, process, and analyze sales data. Currently, we lack a centralized data solution, making it difficult to track key business metrics, optimize performance, and generate insightful reports.

### Solution
We need a **data warehouse** that integrates with **Microsoft Fabric** and allows us to:
- Ingest sales data from external sources
- Organize and structure data efficiently using dimensional modeling
- Ensure fast and accurate reporting on sales performance
- Provide clear visibility into customer behavior, product performance, and revenue trends

### Implementation Summary

#### Data Lakehouse Setup
1. Created workspace: `sales_ws`
2. Created lakehouse: `sales_dlh` for raw data storage
3. Uploaded `sales.csv` and created staging table: `sales_stg`

#### Data Warehouse Setup
1. Created data warehouse: `sales_dwh`
2. Created dedicated schema: `sales_schema`
3. Built star schema with dimension and fact tables
4. Implemented stored procedure for automated ETL
5. Executed analytical queries and created visualizations

## 2. Architecture

```
┌─────────────┐     ┌──────────────────┐     ┌──────────────────┐
│             │     │                  │     │                  │
│  sales.csv  │────>│  Lakehouse       │────>│  Data Warehouse  │
│             │     │  (sales_dlh)     │     │  (sales_dwh)     │
└─────────────┘     │                  │     │                  │
                    │  ┌─────────────┐ │     │  ┌────────────┐  │
                    │  │ sales_stg   │ │     │  │ Dim_       │  │
                    │  │ (staging)   │ │     │  │ Customer   │  │
                    │  └─────────────┘ │     │  └────────────┘  │
                    │                  │     │                  │
                    └──────────────────┘     │  ┌────────────┐  │
                                             │  │ Dim_Item   │  │
                                             │  └────────────┘  │
                                             │                  │
                                             │  ┌────────────┐  │
                                             │  │ Fact_Sales │  │
                                             │  └────────────┘  │
                                             └──────────────────┘
```

### Star Schema Design
- **Fact Table:** `Fact_Sales` - Contains transactional sales data
- **Dimension Tables:**
  - `Dim_Customer` - Customer master data
  - `Dim_Item` - Product master data

## 3. Implementation

### 3.1 Schema Creation

Create a dedicated schema to organize our data warehouse objects logically.

In [3]:
%%sql
-- To organize our data warehouse we will create a dedicated schema 'sales_schema' that will group the tables.

if not exists( select * from sys.schemas where name = 'sales_schema')
begin
exec('create schema sales_schema') -- for creating a schema after using the if not exists check, we need to pass by exec function
end

In [4]:
%%sql
-- Checking available tables in sales_dwh data warehouse 
SELECT *
FROM [sales_dwh].[INFORMATION_SCHEMA].[TABLES]

**Result:** Shows system tables and views in the data warehouse.

!["sales_schema" Default Tables](./Figures/1_sales_schema_default_tables.png)

### 3.2 Table Creation

Create dimensional and fact tables following star schema design principles.

In [26]:
%%sql
-- Next I wil create the Dimensional and fact tables to organize our data for analytics.

-- Create Dim_Customer Table
if not exists (select * from INFORMATION_SCHEMA.TABLES where table_schema ='sales_schema' and table_name = 'Dim_Customer')
CREATE TABLE sales_schema.Dim_Customer (
    CustomerID varchar(255) not null
    ,CustomerName varchar(255)
    ,EmailAddress varchar(255) not null
    --,Constraint PK_Dim_Customer primary key (CustomerID) -- Contraints cannot be declred inside the create in fabric
)

-- Create Dim_Item Table
if not exists (select * from INFORMATION_SCHEMA.TABLES where table_schema = 'sales_schema' and table_name ='Dim_Item')
create table sales_schema.Dim_Item(
    ItemID varchar(255) not null
    ,ItemName varchar(255) not null
)

-- Create Fact_Sales Table 
if not exists (select* from INFORMATION_SCHEMA.TABLES where table_schema = 'sales_schema' and table_name = 'Fact_Sales' )
create table sales_schema.Fact_Sales (
    CustomerID varchar(255) not null
    ,ItemID varchar(255) not null
    ,SalesOrderNumber varchar(30) null
    ,SalesOrderLineNumber int null 
    ,OrderDate date null 
    ,Quantity int null 
    ,TaxAmount float null
    ,UnitPrice float null 
)


UsageError: Cell magic `%%sql` not found.


In [6]:
%%sql
-- Verifying the creation of our dim and fact tables
SELECT *
FROM [sales_dwh].[INFORMATION_SCHEMA].[TABLES]
where table_schema = 'sales_schema'

**Result:** Shows our three tables in the sales_schema.
![Warehouse Tables](./Figures/2_sales_schema_dwh_tables.png)

### 3.3 Constraints & Relationships

Define primary and foreign keys to establish referential integrity.

In [8]:
%%sql
-- Primary & Foreign Keys creation:

-- Defining primary key for Dim_Customer
alter table sales_schema.Dim_Customer
add constraint PK_Dim_Customer primary key Nonclustered (CustomerID) not enforced

-- Defining primary key for Dim_Item
alter table sales_schema.Dim_Item 
add constraint PK_Dim_Item primary key nonclustered (ItemID) not enforced

-- Defining foreign keys for Fact_Sales
alter table sales_schema.Fact_Sales
add constraint FK_Dim_Item foreign key (ItemID) references sales_schema.Dim_Item (ItemID) not enforced

alter table sales_schema.Fact_Sales
add constraint FK_Dim_Customer foreign key (CustomerID) references sales_schema.Dim_Customer(CustomerID) not enforced

### 3.4 Data Loading


In [9]:
%%sql
-- having a look at the data in the staging table Sales_stg that is located in the data lakehouse 
select top 5 *
from sales_dlh.dbo.sales_stg

**Result:** Sample records from the staging table.
![Warehouse Tables](./Figures/3_Sales_stg_table_query.png)

In [13]:
%%sql
-- Load data to the warehouse using a stored procedure

create or alter procedure sales_schema.LoadDataFromStaging (@OrderYear int) as 
begin

-- Insert customers data into Dim_Customer while avoiding duplicates.
insert into sales_schema.Dim_Customer (CustomerID, CustomerName,EmailAddress)
select distinct 
    CustomerName
    ,CustomerName
    ,EmailAddress
from sales_dlh.dbo.sales_stg
where year(OrderDate)=@OrderYear
and not exists (
    select * 
    from sales_schema.Dim_Customer c 
    inner join sales_dlh.dbo.sales_stg s
    on  c.CustomerName = s.CustomerName
    and c.EmailAddress = s.EmailAddress   
    )


-- Insert items data into Dim_Item
insert into sales_schema.Dim_Item(ItemID,ItemName)
select distinct Item,Item
from sales_dlh.dbo.sales_stg
where year(OrderDate) = @OrderYear
and not exists (
    select *
    from sales_schema.Dim_Item i
    inner join sales_dlh.dbo.sales_stg s
    on i.ItemName = s.Item
)

-- insert sales data in Fact_Sales
insert into sales_schema.Fact_Sales(CustomerID,ItemID,SalesOrderNumber,SalesOrderLineNumber,OrderDate,Quantity,TaxAmount,UnitPrice)
select distinct
    cast(CustomerName as varchar(255))
    ,cast(Item as varchar(255))
    ,cast(SalesOrderNumber as varchar(30))
    ,cast(SalesOrderLineNumber as int) 
    ,cast(OrderDate as date) 
    ,cast(Quantity as int) 
    ,cast(TaxAmount as float)
    ,cast(UnitPrice as float)
from sales_dlh.dbo.sales_stg
where year(OrderDate) = @OrderYear

end

GO
-- Executing the procedure
exec sales_schema.LoadDataFromStaging 2021

**Results:**
- Loaded 10,492 customers into `Dim_Customer`
- Loaded 117 products into `Dim_Item`
- Loaded 28,784 transactions into `Fact_Sales`

## 4. Analytics & Insights

### 4.1 Top Customers Analysis

Identify the top 5 customers by total sales revenue in 2021.

In [50]:
%%sql
-- Write a SQL query to retrieve the total sales per customer for the year 2021,
--ensuring the data is grouped by customer and ordered from highest to lowest sales.
select top 5
    c.CustomerName
    ,round(sum(s.Quantity*s.UnitPrice),2) as TotalSales
    
from 
    sales_schema.Dim_Customer c 
    left join sales_schema.Fact_Sales s
    on c.CustomerID = s.CustomerID
where year(s.OrderDate) = 2021
group by c.CustomerName
order by 2 desc


**Results:**

| Customer Name | Total Sales |
|--------------|-------------|
| Jordan Turner | $14,686.70 |
| Nicole Blue | $11,494.93 |
| Maurice Shan | $10,525.60 |
| Janet Munoz | $10,070.11 |
| Alexandra Hall | $9,710.76 |

![Warehouse Tables](./Figures/4_sales_per_customer_query.png)
![Warehouse Tables](./Figures/5_sales_per_customer_chart.png)


### 4.2 Top Products Analysis

Identify the top 5 products by total sales revenue.

In [49]:
%%sql
-- Write a SQL query to retrieve the total sales per product for the year 2021,
-- ensuring the data is grouped by product name and ordered from highest to lowest sales.
select top 5
i.ItemName
,round(sum(s.Quantity * s.UnitPrice),2) as TotalSales

from  sales_schema.Dim_Item i 
left join sales_schema.Fact_Sales s
on i.ItemID = s.ItemID
group by i.ItemName
order by 2 desc


**Results:**

| Item Name | Total Sales |
|-----------|-------------|
| Mountain-200 Black, 46 | $718,987.58 |
| Mountain-200 Silver, 46 | $687,794.17 |
| Mountain-200 Black, 38 | $668,006.02 |
| Mountain-200 Black, 42 | $647,760.93 |
| Mountain-200 Silver, 38 | $641,145.80 |

![Warehouse Tables](./Figures/6_sales_per_product_query.png)

![Warehouse Tables](./Figures/7_sales_per_product_chart.png)

### 4.3 Category Leaders Analysis

Identify the top customer for each product category using window functions.

In [53]:
%%sql
--top_customer_per_product_category_sales
-- Write a SQL query to identify the top customer in terms of total sales for each product category (Helmet, Bike, Gloves) in the year 2021.
-- Ensure the results include the category, customer name, and total sales, ordered from highest to lowest sales.
 WITH CategorizedSales AS (
 SELECT
     CASE
         WHEN i.ItemName LIKE '%Helmet%' THEN 'Helmet'
         WHEN i.ItemName LIKE '%Bike%' THEN 'Bike'
         WHEN i.ItemName LIKE '%Gloves%' THEN 'Gloves'
         ELSE 'Other'
     END AS Category,
     c.CustomerName,
     s.UnitPrice * s.Quantity AS Sales
 FROM sales_schema.Fact_Sales s
 JOIN sales_schema.Dim_Customer c
 ON s.CustomerID = c.CustomerID
 JOIN sales_schema.Dim_Item i
 ON s.ItemID = i.ItemID
 WHERE YEAR(s.OrderDate) = 2021
 ),
 RankedSales AS (
     SELECT
         Category,
         CustomerName,
         SUM(Sales) AS TotalSales,
         ROW_NUMBER() OVER (PARTITION BY Category ORDER BY SUM(Sales) DESC) AS SalesRank
     FROM CategorizedSales
     WHERE Category IN ('Helmet', 'Bike', 'Gloves')
     GROUP BY Category, CustomerName
 )
 SELECT Category, CustomerName, TotalSales
 FROM RankedSales
 WHERE SalesRank = 1
 ORDER BY TotalSales DESC;

**Results:**

| Category | Customer Name | Total Sales |
|----------|--------------|-------------|
| Bike | Carson Butler | $318.00 |
| Helmet | Hailey Patterson | $209.94 |
| Gloves | Joan Coleman | $97.96 |

![Warehouse Tables](./Figures/8_top_customer_per_product_category_sales_query.png)

![Warehouse Tables](./Figures/9_top_customer_per_product_category_sales_chart.png)